In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os 
import numpy 


## Create Folders for Data Prepration

In [3]:
os.mkdir("./train_data")
os.mkdir("./train_data/images")
os.mkdir("./train_data/labels")
os.mkdir("./train_data/images/train")
os.mkdir("./train_data/images/val")
os.mkdir("./train_data/labels/train")
os.mkdir("./train_data/labels/val")

 # *Convert xml files to txt files because yolo expects txt files with normalized bounding boxes* 

In [4]:
import xml.etree.ElementTree as ET
from xml.dom.minidom import parse



path = "../input/face-mask-detection/annotations"
classes = {"with_mask":0,"without_mask":1,"mask_weared_incorrect":2}

for annotations in os.listdir(path):
    
    
    dom = parse(os.path.join(path,annotations))
    root = dom.documentElement
    filename = ".txt".join(root.getElementsByTagName("filename")[0].childNodes[0].data.split(".png"))
    
    image_width = root.getElementsByTagName("width")[0].childNodes[0].data
    image_height = root.getElementsByTagName("height")[0].childNodes[0].data
    with open("./train_data/labels/train/"+filename,"w") as r:
        
        for items in root.getElementsByTagName("object") :
            
            name = items.getElementsByTagName("name")[0].childNodes[0].data
            xmin = items.getElementsByTagName("xmin")[0].childNodes[0].data
            ymin = items.getElementsByTagName("ymin")[0].childNodes[0].data
            xmax = items.getElementsByTagName("xmax")[0].childNodes[0].data
            ymax = items.getElementsByTagName("ymax")[0].childNodes[0].data
            x_center_norm = ((int(xmin)+int(xmax)) / 2 ) / int(image_width)
            y_center_norm = ((int(ymin)+int(ymax))/2) / int(image_height)
            width_norm = ((int(xmax)-int(xmin))/int(image_width))
            height_norm = ((int(ymax)-int(ymin))/int(image_height))
            
            r.write(str(classes[name]))
            r.write(" ")
            r.write(str(x_center_norm))
            r.write(" ")
            r.write(str(y_center_norm))
            r.write(" ")
            r.write(str(width_norm))
            r.write(" ")
            r.write(str(height_norm))
            r.write("\n")
            

# *Check if we  created same  amount of txt files as xml files that are in annotations*

In [5]:
len(os.listdir("./train_data/labels/train/")) == len(os.listdir("../input/face-mask-detection/images"))

# *Copy images to train_data/images/train folder*


In [6]:
import shutil
path = "../input/face-mask-detection/images"
for images in os.listdir(path):
    image_path = os.path.join(path,images)
    shutil.copy(image_path,"./train_data/images/train/"+images)

# *Check if images are equal to labels*

In [7]:
len(os.listdir("./train_data/images/train/")) == len(os.listdir("./train_data/labels/train/"))

# *Shuffle and move to val folders to create train-val split*

In [8]:
from random import shuffle
from glob import glob
import random
import shutil

random.seed(42)
files = glob("./train_data/images/train/*.png")
shuffle(files)
txt_train_path = "./train_data/labels/train/"
img_train_path = "./train_data/images/train/"
txt_val_path = "./train_data/labels/val/"
img_val_path = "./train_data/images/val/"
for i in files[:120]:
    
    filename = i.split("/")[-1]
    shutil.move(txt_train_path+filename.replace(".png",".txt"),txt_val_path+filename.replace(".png",".txt"))
    shutil.move(img_train_path+filename,img_val_path+filename)

# *Check if val labels and val images are equal*

In [9]:
len(os.listdir("./train_data/labels/val/")) == len(os.listdir("./train_data/images/val/"))

# *Create data.yaml file that points to our dataset in a primitive way :)*

In [10]:
with open("/kaggle/working/data.yaml","w") as f:
    f.write("train: ")
    f.write("/kaggle/working/train_data/images/train/")
    f.write("\n")
    f.write("val: ")
    f.write("/kaggle/working/train_data/images/val/")
    f.write("\n")
    f.write("nc: ")
    f.write(" 3")
    f.write("\n")
    f.write("names: ")
    f.write(" ['with_mask','without_mask','mask_weared_incorrect']")
    
    

# *Let's see our data.yaml file* 

In [11]:
with open("/kaggle/working/data.yaml","r") as f:
    print(f.read())

# *Clone Yolov5 repo*

In [12]:
!git clone https://github.com/ultralytics/yolov5

In [13]:
%cd yolov5

In [14]:
!pip install -r requirements.txt

# *Uninstall wandb if you don't need it.Also if you are having problems of interacting with the cell,this is the only work around I found.*

In [15]:
!pip uninstall -q -y wandb 

# *Let's train!*

In [16]:
!python train.py --img 640 --batch 32 --epochs 50 --data /kaggle/working/data.yaml --weights yolov5s.pt --cache 

In [28]:
!cp runs/train/exp/weights/best.pt ../

# *Visualize one of our batch predictions.*

### display detected images
from IPython.display import Image
Image('/kaggle/working/yolov5/runs/train/exp/test_batch1_pred.jpg')

# *Inference on a photo*

In [19]:
!wget https://www.ed.ac.uk/files/styles/landscape_breakpoints_theme_uoe_mobile_1x/public/thumbnails/image/gettyimages-1215069129_jacoblund.png

In [20]:
!python detect.py --source "/kaggle/working/yolov5/gettyimages-1215069129_jacoblund.png" --weights '/kaggle/working/yolov5/runs/train/exp/weights/best.pt'

In [21]:
Image("/kaggle/working/yolov5/runs/detect/exp/gettyimages-1215069129_jacoblund.png")

In [22]:
import os
os.listdir("../train_data/images/val/")

In [25]:
!python detect.py --source "../train_data/images/val/maksssksksss700.png" --weight '/kaggle/working/yolov5/runs/train/exp/weights/best.pt' --name expTestImage --conf 0.4

In [26]:
Image("/kaggle/working/yolov5/runs/detect/expTestImage2/maksssksksss700.png")

# *END !!*